In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
import lsst.sims.maf.db as db
import lsst.sims.maf.metrics as metrics
import lsst.sims.maf.slicers as slicers
import lsst.sims.maf.stackers as stackers
import lsst.sims.maf.plots as plots
import lsst.sims.maf.metricBundles as metricBundles
import numpy as np

/Users/yoachim/lsst/DarwinX86/anaconda/2.2.0/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [2]:
# Set up the database connection
opsdb = db.OpsimDatabase('/Users/yoachim/Scratch/Opsim_sqlites/minion_1016_sqlite.db')
#opsdb = db.OpsimDatabase('/Users/yoachim/Scratch/Opsim_sqlites/opsim3_61_sqlite.db', defaultdbTables={'Summary':['Summary', 'obsHistID']})
outDir = 'astrometry_dcr'
resultsDb = db.ResultsDb(outDir=outDir)

In [3]:
# Look at field 335 and see what the values look like
bundleList = []

slicer = slicers.UserPointsSlicer(1.985364, -1.064323)
metric = metrics.PassMetric()
sql = 'fieldID = 335 and (filter = "r" or filter = "g" or filter = "i" or filter = "z")'
stackerList = []
stackerList.append(stackers.ParallaxFactorStacker())
stackerList.append(stackers.DcrStacker())
stackerList.append(stackers.HourAngleStacker())
bundle = metricBundles.MetricBundle(metric, slicer, sql, stackerList=stackerList)
bundleList.append(bundle)

bundleList.append(metricBundles.MetricBundle(metrics.ParallaxDcrDegenMetric(), slicer, sql, stackerList=stackerList))
#bundleList.append(metricBundles.MetricBundle(metrics.ParallaxHADegenMetric(), slicer, sql, stackerList=stackerList))

# bundleList.append(metricBundles.MetricBundle(metrics.ParallaxDcrDegenMetric(seeingCol='seeing', m5Col='fivesigma_modified'), slicer, sql, stackerList=stackerList))

bundleDict = metricBundles.makeBundlesDictFromList(bundleList)
bgroup = metricBundles.MetricBundleGroup(bundleDict, opsdb, outDir=outDir, resultsDb=resultsDb)
bgroup.runAll()

Querying database with constraint fieldID = 335 and (filter = "r" or filter = "g" or filter = "i" or filter = "z")
Found 680 visits
Running:  ['opsim_Pass_fieldID_335_and_r_or_g_or_i_or_z_USER', 'opsim_ParallaxDcrDegenMetric_fieldID_335_and_r_or_g_or_i_or_z_USER']
Completed metric generation.
Running reduce methods.
Running summary statistics.
Completed.


/Users/yoachim/gitRepos/sims_maf/python/lsst/sims/maf/stackers/baseStacker.py:122: UserWarning: Warning - column HA already present in simData, will be overwritten.
  %(col))


In [9]:
bundle.metricValues[0].dtype

dtype([('obsHistID', '<i8'), ('FWHMgeom', '<f8'), ('fieldRA', '<f8'), ('altitude', '<f8'), ('fieldDec', '<f8'), ('fiveSigmaDepth', '<f8'), ('filter', 'S256'), ('lst', '<f8'), ('expMJD', '<f8'), ('ra_pi_amp', '<f8'), ('dec_pi_amp', '<f8'), ('ra_dcr_amp', '<f8'), ('dec_dcr_amp', '<f8'), ('zenithDistance', '<f8'), ('PA', '<f8'), ('HA', '<f8')])

In [12]:
print 'Looking at single point in sky at ra=%f, dec=%f' % (slicer.ra[0], slicer.dec[0])

Looking at single point in sky at ra=1.985364, dec=-1.064323


In [7]:
print 'mjd, parallax_ra (arcsec), parallax_dec (arcsec), dcr_ra (arcsec), dcr_dec (arcsec)'
for vals in bundle.metricValues[0]:
    print '%f, %f, %f, %f, %f' % (vals['expMJD'], vals['ra_pi_amp'], vals['dec_pi_amp'], 
                                 vals['ra_dcr_amp'], vals['dec_dcr_amp'])


mjd, parallax_ra (arcsec), parallax_dec (arcsec), dcr_ra (arcsec), dcr_dec (arcsec)
59581.360061, 0.163033, -0.969898, -0.040768, -0.017396
59582.348523, 0.146190, -0.972389, -0.041626, -0.020836
59599.065319, -0.141392, -0.969662, 0.042055, -0.023544
59599.084148, -0.141714, -0.969611, 0.042245, -0.032261
59601.195992, -0.177720, -0.963274, -0.016401, -0.072644
59601.209974, -0.177957, -0.963227, -0.025913, -0.067640
59609.033058, -0.308941, -0.928269, 0.041725, -0.021454
59609.048726, -0.309198, -0.928181, 0.042393, -0.028358
59631.130315, -0.640398, -0.738770, -0.030318, -0.074064
59659.125223, -0.921653, -0.348808, -0.045377, -0.028350
59659.140678, -0.921753, -0.348561, -0.044377, -0.021290
59687.052435, -0.992684, 0.119417, -0.042224, -0.024819
59687.068800, -0.992660, 0.119694, -0.040967, -0.018089
59842.368134, 0.893286, 0.391880, 0.046045, -0.014264
59842.382621, 0.893384, 0.391653, 0.048346, -0.020180
59853.334361, 0.951443, 0.213813, 0.063501, -0.018035
59853.344734, 0.95148